In [2]:
!pip install datasets keras-tuner contractions pyspellchecker langdetect deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.6 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 88.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 8.4 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=0b9ffbbf318e1272a64e05ef578b0784a8a95c0369a221f681cf8558e430ee0d
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [3]:
import numpy as np

import torch

import torch.nn as nn

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Dropout, Embedding, Bidirectional, GRU, Layer, Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D, LSTM, Concatenate, MultiHeadAttention, Add,LayerNormalization, BatchNormalization

from tensorflow.keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from datasets import load_dataset

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, accuracy_score

import re

import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow import keras

import keras_tuner as kt

from torch.optim import AdamW

from torch.optim.lr_scheduler import ReduceLROnPlateau

from tqdm import tqdm

from transformers import BertTokenizer, BertModel

from torch.utils.data import DataLoader

from deep_translator import MyMemoryTranslator

from langdetect import detect

import spacy

import contractions

import nltk

from spellchecker import SpellChecker

import random

from GPUtil import showUtilization as gpu_usage

from numba import cuda

from datasets import load_dataset

import optuna

from tensorflow.keras.models import Model

from tensorflow.keras.regularizers import l2

from sklearn.linear_model import LogisticRegression

# Set a seed value

seed_value = 42



random.seed(seed_value)

# For NumPy

np.random.seed(seed_value)



# For TensorFlow

tf.random.set_seed(seed_value)



# For PyTorch

torch.manual_seed(seed_value)

if torch.cuda.is_available():

    torch.cuda.manual_seed(seed_value)

    torch.cuda.manual_seed_all(seed_value)  # if using multi-GPU

    torch.backends.cudnn.deterministic = True

    torch.backends.cudnn.benchmark = False

# Dataset Preparation

In [34]:
# Load the dataset

dataset = load_dataset("rotten_tomatoes")



# Split the dataset into training, validation, and test sets

train_dataset = dataset['train']

validation_dataset = dataset['validation']

test_dataset = dataset['test']

# Generate Word Embeddings

In [35]:
# Download glove and unzip it in Notebook.

!wget http://nlp.stanford.edu/data/glove.6B.zip

!unzip glove*.zip

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


--2024-11-07 19:05:38--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-11-07 19:05:39--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-11-07 19:05:39--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: 'glove.6B.zip'

glov

# Data Exploration & Preprocessing

In [37]:
train_text = train_dataset['text']



# Initialize the tokenizer

tokenizer = Tokenizer()

tokenizer.fit_on_texts(train_text)



# Number of unique words in the train dataset

vocab_size = len(tokenizer.word_index)

print("Vocabulary size of train data:", vocab_size)

Vocabulary size of train data: 17451


In [38]:
def load_glove_vocab(filepath):

    glove_vocab = set()

    with open(filepath, encoding="utf8") as f:

        for line in f:

            word, *vector = line.split()

            glove_vocab.add(word)

    return glove_vocab



# Load Glove words into a set

glove_vocab = load_glove_vocab('glove.6B.300d.txt')



# Identify OOV words (Words appeared in training data but not in the GloVe dictionary)

oov_words = []

for word in tokenizer.word_index:

  if word not in glove_vocab:

    oov_words.append(word)



# Print OOV words

print("Size of Glove Vocabulary:", len(glove_vocab))

print("Out-Of-Vocabulary Words:", oov_words)

print("Number of OOV words:", len(oov_words))

Size of Glove Vocabulary: 400000
Out-Of-Vocabulary Words: ["it's", "doesn't", "there's", "that's", "isn't", "don't", "can't", "film's", "you're", "you'll", "he's", "movie's", "won't", "what's", "you've", "i'm", "didn't", "they're", "year's", '\x96', "you'd", "aren't", "i've", "we've", "couldn't", "she's", "man's", "we're", "wasn't", "i'd", '\x97', "who's", "director's", "haven't", "here's", "story's", "characters'", "wouldn't", "i'll", "'the", "woman's", "hasn't", "world's", "filmmaker's", "children's", "moore's", "one's", "soderbergh's", "america's", "disney's", "shouldn't", "ain't", "character's", "cinema's", "women's", "cho's", "hoffman's", "kids'", "today's", "amy's", "wilde's", "life's", "they'll", "emperor's", "sandler's", "scorsese's", "allen's", "it'll", "filmmakers'", 'cletis', "carvey's", "let's", "stevenson's", 'waydowntown', "polanski's", "he'd", "weren't", "lee's", "everyone's", "parker's", "woo's", "'i", "greene's", "2002's", "actor's", "bullock's", "writer's", "shakespea

Cleaning Training Text

In [40]:
def expand_contractions(text):

    return contractions.fix(text)

In [41]:
def re_manip(text):

    #remove apostrophe and punctuations

    text = re.sub(r"'\S*", "", text)

    text = re.sub(r'[^\w\s]', ' ', text)

    

    #remove numbers

    text = re.sub(r'\b\d+(st|nd|rd|th)?\b', '', text)

    

    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [42]:
### INTERMEDIARY STEP TO NOT OVERUSE TRANSLATION API

cleaned_train_text = [re_manip(expand_contractions(text)) for text in train_text]



tokenizer = Tokenizer(oov_token='<UNK>')  # <UNK> for unseen words

tokenizer.fit_on_texts(cleaned_train_text)



# Identify OOV words (Words appeared in training data but not in the GloVe dictionary)

oov_words = []

for word in tokenizer.word_index:

  if word not in glove_vocab:

    oov_words.append(word)

In [43]:
language_locale_map = {

    "es": "es-ES",

    "fr": "fr-FR",

    "de": "de-DE",

    "it": "it-IT",

    "pt": "pt-PT",

    "ru": "ru-RU",

    "zh": "zh-CN",

    "ja": "ja-JP",

    "ko": "ko-KR",

    "ar": "ar-SA"

}

In [44]:
spell = SpellChecker(distance=3)

def correct_and_translate(word):

    spell.distance = 3

    corrected_word = spell.correction(word) or word

    try:

        lang = detect(corrected_word)

        if lang in language_locale_map and lang != 'en':

            source_locale = language_locale_map[lang]

            translator = MyMemoryTranslator(source=source_locale, target='en-US')

            corrected_word = translator.translate(corrected_word)

    except Exception as e:

        print(f"Error translating word '{corrected_word}': {e}")

    

    return corrected_word

In [45]:
nlp = spacy.load("en_core_web_sm")

spell = SpellChecker(distance=3)

def lemmatize_with_handling(text, oov_words):

    doc = nlp(text)

    lemmatized_words = []

    for token in doc:

        if token.text in oov_words:

            lemmatized_words.append(correct_and_translate(token.text))

        else:

            spell.distance = 2

            lemma = token.lemma_

            if token.lemma_ not in glove_vocab:

                lemma = spell.correction(lemma) or lemma

            lemmatized_words.append(lemma)

    return " ".join(lemmatized_words)



cleaned_train_text = [lemmatize_with_handling(text, oov_words) for text in cleaned_train_text]



print("Before:", train_text[0])

print("After:", cleaned_train_text[0])

Before: the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .
After: the rock be destine to be the century new conan and that he be go to make a splash even great than arnold schwarzenegger jean claud van damme or steven segal


In [46]:
### FUNCTION PURELY FOR TEST/VAL DATA

def clean_words(texts):

    cleaned_text = [re_manip(expand_contractions(text)) for text in texts]

    tokenizer.fit_on_texts(cleaned_text)  

    # Identify OOV words (Words appeared in training data but not in the GloVe dictionary)

    oov_word = []

    for word in tokenizer.word_index:

      if word not in glove_vocab:

        oov_word.append(word)

    finalised_text = [lemmatize_with_handling(text, oov_word) for text in cleaned_text]

    return finalised_text

In [47]:
#Clean test/validation sets

test_text = test_dataset['text']

val_text = validation_dataset['text']

cleaned_test_text = clean_words(test_text)

cleaned_val_text = clean_words(val_text)

Assigning 'UNK' Label on Cleaned Training Text

In [48]:
# Initialise the tokeniser

tokenizer = Tokenizer(oov_token='<UNK>')  # <UNK> for unseen words

tokenizer.fit_on_texts(cleaned_train_text)



# Identify OOV words (Words appeared in training data but not in the GloVe dictionary)

oov_words = []

for word in tokenizer.word_index:

  if word not in glove_vocab:

    oov_words.append(word)



# Print OOV words

print("Out-of-vocabulary words:", oov_words)

print("Number of OOV words:", len(oov_words))

Out-of-vocabulary words: ['<UNK>', 'exhilarate', 'engross', 'moviemake', 'stylize', 'unclinch', 'windtalker', 'clockstopper', 'enthral', 'waydowntown', 'ozpetek', 'everlaste', 'spellbind', 'kosashvili', 'tambac', 'swinge', 'feardotcom', 'overstuff', 'deprave', 'watstein', 'appal', 'breathtake', 'throe', 'rollick', 'tatter', 'runteldat', 'misconceive', 'cheerly', 'janklowicz', 'frissons', 'infatuate', 'roteirista', 'stupefy', 'groundbreake', 'cliffsnote', 'wertmull', 'wisegirls', 'enrapturing', 'originalidad', 'overpraise', 'syncopate', 'obviation', 'gorefests', 'makmalbaf', 'shapelessly', 'mullinski', 'narcotizing', 'sparklingly', 'dreyfu', 'nonthreatene', 'dominatrixes', 'denlopp', 'sappier', 'sorprenderá', 'sarcastic\xa0', 'powaqqatsi', 'kaputschnik', 'monkeyfun', 'bierbichler', 'unindicte', 'datedness', 'inhospitality', 'hastier', 'existência', 'inquestionável', 'hotdogging', 'sogginess', 'stuffiest', 'limewater', 'muckrake', 'premiss', 'culminant', 'desfecho', 'lentamente', 'soarin

# Model Training

In [49]:
# Convert texts to sequences (words to the word index in tokenizer)
train_sequences = tokenizer.texts_to_sequences(cleaned_train_text)

# Pad the sequences (ensure they are all the same length)
max_seq_len = 100
tokenizer.word_index['<PAD>'] = 0
train_padded = pad_sequences(train_sequences, maxlen=max_seq_len, padding='post', value=tokenizer.word_index['<PAD>'])

print("Before:", train_sequences[0])
print("After:", list(train_padded[0]))

Before: [2, 579, 3, 2602, 7, 3, 2, 748, 98, 4251, 5, 10, 61, 3, 71, 7, 25, 4, 2291, 60, 113, 34, 1349, 1854, 1855, 7481, 1439, 5371, 41, 834, 7482]
After: [2, 579, 3, 2602, 7, 3, 2, 748, 98, 4251, 5, 10, 61, 3, 71, 7, 25, 4, 2291, 60, 113, 34, 1349, 1854, 1855, 7481, 1439, 5371, 41, 834, 7482, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [50]:
# Prepare inputs for model
# Convert validation and test texts to sequences
val_sequences = tokenizer.texts_to_sequences(cleaned_val_text)
test_sequences = tokenizer.texts_to_sequences(cleaned_test_text)

# Pad sequences
val_padded = pad_sequences(val_sequences, maxlen=max_seq_len, padding='post', value=tokenizer.word_index['<PAD>'])
test_padded = pad_sequences(test_sequences, maxlen=max_seq_len, padding='post', value=tokenizer.word_index['<PAD>'])

# Labels
train_labels = np.array(train_dataset['label'])
val_labels = np.array(validation_dataset['label'])
test_labels = np.array(test_dataset['label'])

In [51]:
# word_index refers to the vocab in training data?
print(tokenizer.word_index)
print(len(tokenizer.word_index))

{'<UNK>': 1, 'the': 2, 'be': 3, 'a': 4, 'and': 5, 'of': 6, 'to': 7, 'it': 8, 'in': 9, 'that': 10, 'film': 11, 'not': 12, 'as': 13, 'but': 14, 'have': 15, 'movie': 16, 'with': 17, 'this': 18, 'for': 19, 'an': 20, 'its': 21, 'you': 22, 'on': 23, 'do': 24, 'make': 25, 'by': 26, 'one': 27, 'like': 28, 'about': 29, 'more': 30, 'i': 31, 'at': 32, 'from': 33, 'than': 34, 'all': 35, 'his': 36, 'good': 37, 'so': 38, 'will': 39, 'well': 40, 'or': 41, 'if': 42, 'story': 43, 'we': 44, 'character': 45, 'can': 46, 'what': 47, 'they': 48, 'time': 49, 'there': 50, 'who': 51, 'just': 52, 'too': 53, 'out': 54, 'up': 55, 'into': 56, 'comedy': 57, 'no': 58, 'most': 59, 'even': 60, 'he': 61, 'much': 62, 'work': 63, 'see': 64, 'get': 65, 'bad': 66, 'performance': 67, 'some': 68, 'director': 69, 'way': 70, 'go': 71, 'feel': 72, 'would': 73, 'only': 74, 'funny': 75, 'look': 76, 'life': 77, 'their': 78, 'little': 79, 'take': 80, 'love': 81, 'your': 82, 'very': 83, 'enough': 84, 'when': 85, 'come': 86, 'never':

In [52]:
def create_vector_matrix(filepath, word_index, embedding_dim):
    # Calculate the size of the vocabulary (including UNK token)
    vocab_size = len(word_index)
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    # Load GloVe vectors from the file
    glove_embeddings = {}
    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            glove_embeddings[word] = np.array(vector, dtype=np.float32)

    # Create the embedding matrix
    for word, idx in word_index.items():
        if word in glove_embeddings:
            embedding_matrix[idx] = glove_embeddings[word]
        else:
            # Set the embedding to zero if word is OOV (including <UNK>)
            embedding_matrix[idx] = np.zeros(embedding_dim)


    return torch.tensor(embedding_matrix, dtype=torch.float32)


# Example usage with GloVe 100-dimensional embeddings
embedding_dim = 300
embedding_matrix = create_vector_matrix('glove.6B.300d.txt', tokenizer.word_index, embedding_dim)


# No. of rows should be equal to vocab size, no. of columns should be equal to vector dimension
print("Embedding Matrix Shape=> ", embedding_matrix.shape)

# Print the dense vector for the UNK token
unk_idx = tokenizer.word_index["<UNK>"]
print("Dense vector for UNK token is => ", embedding_matrix[unk_idx])

Embedding Matrix Shape=>  torch.Size([13257, 300])
Dense vector for UNK token is =>  tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

# Ensemble Learning (Stacked ensemble using Logistic Regression)

In [58]:
#CNN Model

from tensorflow.keras import regularizers
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

vocab_size = len(tokenizer.word_index)

modelCNN = Sequential()
modelCNN.add(tf.keras.layers.Embedding(input_dim=vocab_size,
                    output_dim=embedding_dim,
                    input_shape=(max_seq_len,),
                    weights=[embedding_matrix],
                    trainable=True,
                    embeddings_regularizer = regularizers.l2(0.0005)))

modelCNN.add(tf.keras.layers.Conv1D(128,3, activation='relu',\
                                 kernel_regularizer = regularizers.l2(0.0005),\
                                 bias_regularizer = regularizers.l2(0.0005)))

modelCNN.add(tf.keras.layers.GlobalMaxPooling1D())

#modelCNN.add(tf.keras.layers.Dense(4, activation='sigmoid',\
#                                kernel_regularizer=regularizers.l2(0.001)))
modelCNN.add(tf.keras.layers.Dropout(0.5))
modelCNN.add(tf.keras.layers.Dense(1, activation='sigmoid'))
modelCNN.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)
modelCNN.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 100, 300)       │     3,977,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 98, 128)        │       115,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_2          │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,092,557 (15.61 MB)

 Trainable params: 4,092,557 (15.61 MB)

 Non-trainable params: 0 (0.00 B)

In [61]:
# biLSTM model
bilstm_model = Sequential()
vocab_size = len(tokenizer.word_index)
# Add the Embedding layer with GloVe weights and input shape
bilstm_model.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_dim,
                    input_shape=(max_seq_len,),
                    weights=[embedding_matrix],
                    trainable=True))  # Set trainable=False to freeze GloVe weights, but we are updating the word embeddings, hence trainable= True

# Add a Bidirectional LSTM layer (2 layers currently)
bilstm_model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
bilstm_model.add(Bidirectional(LSTM(units=64, return_sequences=False)))
bilstm_model.add(Dropout(0.3))

# Add a Dense layer for classification
bilstm_model.add(Dense(32, activation='relu'))
bilstm_model.add(Dropout(0.3))
bilstm_model.add(Dense(1, activation='sigmoid'))

adam = Adam(learning_rate=0.001)
bilstm_model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

bilstm_model.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 100, 300)       │     3,977,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 100, 256)       │       439,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 128)            │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,584,909 (17.49 MB)

 Trainable params: 4,584,909 (17.49 MB)

 Non-trainable params: 0 (0.00 B)

In [67]:
# Train the CNN model
cnn_history=modelCNN.fit(train_padded, train_labels, validation_data=(val_padded, val_labels),
          epochs=30, batch_size=32)


# Train the biLSTM model with early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)
bilstm_history = bilstm_model.fit(train_padded, train_labels, validation_data=(val_padded, val_labels),
          epochs=10, batch_size=32, callbacks=[early_stopping])

Epoch 1/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9920 - loss: 0.1670 - val_accuracy: 0.7355 - val_loss: 0.9771
Epoch 2/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9944 - loss: 0.1628 - val_accuracy: 0.7373 - val_loss: 1.0050
Epoch 3/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9955 - loss: 0.1539 - val_accuracy: 0.7411 - val_loss: 1.0128
Epoch 4/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9930 - loss: 0.1563 - val_accuracy: 0.7411 - val_loss: 1.0355
Epoch 5/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9930 - loss: 0.1510 - val_accuracy: 0.7458 - val_loss: 1.0298
Epoch 6/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9938 - loss: 0.1555 - val_accuracy: 0.7467 - val_loss: 1.0135
Epoch 7/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9937 - loss: 0.1549 - val_accuracy: 0.7439 - val_loss: 1.0558
Epoch 8/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9940 - loss: 0.1501 - val_accuracy: 0.

In [71]:
# Get predictions from both models on training data
cnn_train_preds = modelCNN.predict(train_padded)
bilstm_train_preds = bilstm_model.predict(train_padded)

# Stack predictions as features
train_stack = np.hstack((cnn_train_preds, bilstm_train_preds))

# Train Logistic Regression meta model
meta_model = LogisticRegression()
meta_model.fit(train_stack, train_labels)

# Test set predictions
cnn_test_preds = modelCNN.predict(test_padded)
bilstm_test_preds = bilstm_model.predict(test_padded)

# Stack test predictions
test_stack = np.hstack((cnn_test_preds, bilstm_test_preds))

# Meta-model predictions
ensemble_preds = meta_model.predict(test_stack)


267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
267/267 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [74]:
# Individual model accuracies
cnn_test_loss, cnn_test_accuracy = modelCNN.evaluate(test_padded, test_labels, verbose=0)
bilstm_test_loss, bilstm_test_accuracy = bilstm_model.evaluate(test_padded, test_labels, verbose=0)

# Ensemble model predictions and accuracy
ensemble_preds = meta_model.predict(test_stack)
ensemble_accuracy = accuracy_score(test_labels, ensemble_preds)

# Display results
print("\nModel Performance Summary:")
print(f"CNN Model Test Accuracy: {cnn_test_accuracy:.4f}")
print(f"BiLSTM Model Test Accuracy: {bilstm_test_accuracy:.4f}")
print(f"Ensemble Model Test Accuracy: {ensemble_accuracy:.4f}")



Model Performance Summary:
CNN Model Test Accuracy: 0.7345
BiLSTM Model Test Accuracy: 0.7992
Ensemble Model Test Accuracy: 0.7430
